In [1]:
from pipeline import DataProcessing

c:\Users\krodriguez0109\AppData\Local\anaconda3\envs\pytorch\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\krodriguez0109\AppData\Local\anaconda3\envs\pytorch\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
c:\Users\krodriguez0109\AppData\Local\anaconda3\envs\pytorch\lib\site-packages\numpy\.libs\libopenblas64__v0.3.23-246-g3d31191b-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
dp = DataProcessing('./data/coyote_0731_1000.xlsx','TotalCostUSD','TotalCostUSD','max')

In [3]:
cleandf = dp.pipeline()

Loading Data...
Getting Size...
Data size: (1000, 99)
Summarizing Type...
Data types summary:
 int64      39
float64    29
bool       20
object     11
Name: count, dtype: int64
Summarizing Missing Data...
Missing data summary:
CommodityCode    622
dtype: int64


Removing Duplicates...
Handling Missing Data...
# rows dropped with missing values in the target variable: 0
Dropped columns: ['CommodityCode']
Filled missing values using mode
Missing data summary:
Series([], dtype: int64)
Detecting Outliers...
24 outliers detected in Mode


26 outliers detected in ProgressType


27 outliers detected in Plan


44 outliers detected in ShipmentType


...

22 outliers detected in LoadDate-Day


1 outliers detected in DelDate-Day
Encoding Features...
Analyzing Data...


Selecting Features...


  0%|          | 0/20 [00:00<?, ?it/s]

ExplainerError: Additivity check failed in TreeExplainer! Please ensure the data matrix you passed to the explainer is the same shape that the model was trained on. If your data shape is correct then please report this on GitHub. Consider retrying with the feature_perturbation='interventional' option. This check failed because for one of the samples the sum of the SHAP values was 8139.600098, while the model output was 8139.594727. If this difference is acceptable you can set check_additivity=False to disable this check.

In [ ]:
cleandf

In [4]:
from xgboost import XGBRegressor
from BorutaShap import BorutaShap
import pandas as pd

In [29]:
# df = pd.read_csv('./bordata.csv',parse_dates=True)
df = pd.read_excel('./data/coyote_0731_1000.xlsx',parse_dates=True)
# df = df.iloc[: , 1:]
df = df.select_dtypes(include='number')
df = df[df.columns.drop(list(df.filter(regex='ID')))]

In [15]:
df

,TotalCostUSD,Mode,ProgressType,Type,StateType,LoadingPosition,Plan,ShipmentType,EquipmentLength,EquipmentWidth,...,TRUCKD11,DJTA,DTB3,GASREGW,IPG2122N,IPG3361T3S,GDP,PPIACO,BUSINV,PCU48414841
0,423.82,2,8,1,1,0,0,1,53,0,...,115.9,13738.04,4.2,3.239,91.7098,102.9025,26137.992,257.897,2545624,207.137
1,1300.00,1,8,1,1,2,0,1,53,0,...,115.9,13738.04,4.2,3.239,91.7098,102.9025,26137.992,257.897,2545624,207.137
2,300.00,1,8,2,1,2,0,5,48,0,...,115.9,13738.04,4.2,3.239,91.7098,102.9025,26137.992,257.897,2545624,207.137
3,873.00,1,8,1,1,2,0,1,48,0,...,115.9,13738.04,4.2,3.239,91.7098,102.9025,26137.992,257.897,2545624,207.137
4,267.48,2,8,1,1,0,0,1,53,0,...,115.9,13738.04,4.2,3.239,91.7098,102.9025,26137.992,257.897,2545624,207.137
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,1200.00,1,8,1,1,2,0,0,53,0,...,115.9,13738.04,4.2,3.239,91.7098,102.9025,26137.992,257.897,2545624,207.137
996,2800.00,1,8,1,1,2,0,1,53,0,...,115.9,13738.04,4.2,3.239,91.7098,102.9025,26137.992,257.897,2545624,207.137
997,2600.00,1,8,1,1,2,0,1,18,0,...,115.9,13738.04,4.2,3.239,91.7098,102.9025,26137.992,257.897,2545624,207.137
998,509.51,1,8,1,1,2,0,0,53,102,...,115.9,13738.04,4.2,3.239,91.7098,102.9025,26137.992,257.897,2545624,207.137


In [30]:
feature_selection_model = XGBRegressor()
Feature_Selector = BorutaShap(model=feature_selection_model,
                              importance_measure='shap',
                              classification=False
                              )

Feature_Selector.fit(X=df.drop('TotalCostUSD',axis=1), y=df['TotalCostUSD'], n_trials=20, sample=True,
                     train_or_test = 'train', normalize=False,
                     verbose=False)

  0%|          | 0/20 [00:00<?, ?it/s]